# EDA for Prototype Fire Data
#### Hannah Catri 217569841




In [1]:
import pandas as pd
from sodapy import Socrata
import datetime
from pandas_profiling import ProfileReport
import requests
import json
import geopandas as gpd
import plotly.express as px
import datetime

## Emergency Services Data Stream

#### Fire Department Calls for Service

In [2]:
# Get the number of rows in the dataset
# Save as a variable in case paging method of data retireval is needed
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)

The number of incidents in the set is 5669231


In [3]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
data=pd.read_json(query_str)
print(data.shape)
data.sample(5)

The number of incidents in the set is 5170
(5170, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
2840,212801871,E32,21121759,Medical Incident,2021-10-07T00:00:00.000,2021-10-07T00:00:00.000,2021-10-07T14:00:04.000,2021-10-07T14:02:58.000,2021-10-07T14:03:09.000,2021-10-07T14:04:21.000,...,1,ENGINE,1,9.0,11,Excelsior,212801871-E32,"{'type': 'Point', 'coordinates': [-122.4272661...",NaN,NaN
1490,212781089,70,21120853,Medical Incident,2021-10-05T00:00:00.000,2021-10-05T00:00:00.000,2021-10-05T10:11:26.000,2021-10-05T10:12:30.000,2021-10-05T10:12:46.000,2021-10-05T10:12:50.000,...,1,MEDIC,1,4,5,Japantown,212781089-70,"{'type': 'Point', 'coordinates': [-122.4274902...",2021-10-05T10:34:33.000,2021-10-05T10:39:45.000
3657,212811848,54,21122190,Medical Incident,2021-10-08T00:00:00.000,2021-10-08T00:00:00.000,2021-10-08T13:45:15.000,2021-10-08T13:45:15.000,2021-10-08T13:46:04.000,2021-10-08T13:46:11.000,...,1,MEDIC,1,2,6,Tenderloin,212811848-54,"{'type': 'Point', 'coordinates': [-122.4179014...",2021-10-08T14:06:09.000,2021-10-08T14:12:00.000
1207,212782498,KM10,21121019,Medical Incident,2021-10-05T00:00:00.000,2021-10-05T00:00:00.000,2021-10-05T16:49:19.000,2021-10-05T16:49:19.000,2021-10-05T16:49:39.000,2021-10-05T16:50:13.000,...,1,PRIVATE,2,1,3,North Beach,212782498-KM10,"{'type': 'Point', 'coordinates': [-122.4074306...",2021-10-05T17:18:52.000,2021-10-05T17:29:18.000
1359,212781777,E01,21120935,Alarms,2021-10-05T00:00:00.000,2021-10-05T00:00:00.000,2021-10-05T13:21:43.000,2021-10-05T13:22:59.000,2021-10-05T13:23:09.000,2021-10-05T13:24:05.000,...,1,ENGINE,1,3,6,South of Market,212781777-E01,"{'type': 'Point', 'coordinates': [-122.3996035...",NaN,NaN


In [3]:
%%time
# Configure the Socrata client and create the generator that will retrieve the data
client = Socrata("data.sfgov.org", None)
dataset_gen=client.get("nuek-vuh3", limit=2000)
# Extract the dataset, this may take a while.
# Use shape to confirm the size matches the rows as per above maxNumber.
fire_cfs_df = pd.DataFrame.from_records(dataset_gen)
fire_cfs_df.shape
fire_cfs_df.sample(5)

CPU times: user 127 ms, sys: 13.3 ms, total: 141 ms
Wall time: 946 ms


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,:@computed_region_ajp5_b2md,on_scene_dttm,transport_dttm,hospital_dttm
199,001340155,T18,00039914,Alarms,2000-05-13T00:00:00.000,2000-05-13T14:35:09.000,2000-05-13T14:29:54.000,2000-05-13T14:35:09.000,2000-05-13T14:35:35.000,2000-05-13T14:37:36.000,...,3,8,4,Sunset/Parkside,001340155-T18,"{'type': 'Point', 'coordinates': [-122.4977704...",35,NaN,NaN,NaN
861,013000030,T07,01091904,Other,2001-10-27T00:00:00.000,2001-10-26T03:35:59.000,2001-10-27T03:35:58.000,2001-10-27T03:35:59.000,2001-10-27T03:35:58.000,NaN,...,2,6,9,Mission,013000030-T07,"{'type': 'Point', 'coordinates': [-122.4176589...",20,NaN,NaN,NaN
1624,021690041,T11,02050051,Other,2002-06-18T00:00:00.000,2002-06-17T03:03:41.000,2002-06-18T03:03:23.000,2002-06-18T03:03:41.000,2002-06-18T03:03:47.000,2002-06-18T03:06:20.000,...,1,6,9,Mission,021690041-T11,"{'type': 'Point', 'coordinates': [-122.4173830...",20,2002-06-18T03:08:44.000,NaN,NaN
1444,020500035,E36,02014790,Medical Incident,2002-02-19T00:00:00.000,2002-02-18T04:13:22.000,2002-02-19T04:13:03.000,2002-02-19T04:13:22.000,2002-02-19T04:14:11.000,NaN,...,2,2,6,Tenderloin,020500035-E36,"{'type': 'Point', 'coordinates': [-122.4184811...",36,NaN,NaN,NaN
1323,020810300,T07,02024301,Structure Fire,2002-03-22T00:00:00.000,2002-03-22T17:45:32.000,2002-03-22T17:45:31.000,2002-03-22T17:45:32.000,2002-03-22T17:45:37.000,2002-03-22T17:47:10.000,...,2,6,9,Mission,020810300-T07,"{'type': 'Point', 'coordinates': [-122.4140860...",20,NaN,NaN,NaN


In [6]:
profile = ProfileReport(data, title="Fire Dept Calls for Service")
profile.to_widgets()

Summarize dataset:   0%|          | 0/47 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Response Times

In [4]:
# Select relevant data needed for aggregation
cols=['dispatch_dttm','on_scene_dttm','battalion','original_priority','unit_type' ]
responseTimes=data[cols].copy()
# Clean data - remove empty values
responseTimes.dropna(inplace=True)
# Clean data - convert data to date/time
responseTimes['dispatch_dttm']= pd.to_datetime(responseTimes['dispatch_dttm'])
responseTimes['on_scene_dttm']= pd.to_datetime(responseTimes['on_scene_dttm'])
# Calculation - time from dispatch to on scene
responseTimes['ResponseMins']=((responseTimes.on_scene_dttm - responseTimes.dispatch_dttm).dt.total_seconds())/60
responseTimes.head(3)

,dispatch_dttm,on_scene_dttm,battalion,original_priority,unit_type,ResponseMins
0,2021-10-04 02:34:51,2021-10-04 02:43:15,B03,2,TRUCK,8.400000
1,2021-10-04 19:58:42,2021-10-04 20:01:10,B05,3,CHIEF,2.466667
2,2021-10-04 12:51:22,2021-10-04 12:52:47,B02,E,TRUCK,1.416667


In [5]:
# Aggregation - Response time by day
df = responseTimes.set_index('dispatch_dttm').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')
df.head(3)

,ResponseMins
dispatch_dttm,
2021-10-04,7.674911
2021-10-05,7.379866
2021-10-06,7.639600


In [14]:
fig = px.line(df,title = 'Avg Response Time Past Week')
# GREY = AGGREGATE DATA
fig.update_layout(paper_bgcolor="#e0e0e0",
                 font_color="black",
                 xaxis_title="Date (Prior 10 days)",
                  yaxis_title="Average Response Time (Minutes)")

fig.show()

#### Tweets with the word 'fire' in region

In [ ]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMdDTQEAAAAAfYqWjo5%2BOgmFP%2FmqGGkq74pEahs%3DNqy7E6BdRittRWu2v3DJAVXEvbDLhAejdCBccR4mQPNO70s9Sl'

In [ ]:

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    return headers

In [ ]:
def create_url(keyword,start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=2)).isoformat()
#Inputs for the request
headers = create_headers(bearer_token)
keyword = "caldor fire lang:en"
#place = "San Francisco"
start_time = periodstart[:-2]+'Z'
end_time = periodend[:-2]+'Z'
max_results = 100

In [ ]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

In [ ]:
tweets=pd.json_normalize(json_response['data'])
tweets.head()

In [ ]:
profile = ProfileReport(tweets, title="Fire Related Tweets")
profile.to_widgets()

## Climate Data Stream

#### Weather API

Add weather API: https://www.weather.gov/documentation/services-web-api
